In [ ]:
# 필요한 모듈 불러오기
import pandas as pd
import numpy as np
from os.path import join

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns

In [ ]:
# train 데이터와 test 데이터 경로를 지정한다.
data_dir = 'C:/Users/ZAKAR/Documents/GitHub/AIFFEL/Exploration/Quest12_EX02/data'
train_data_path = join(data_dir, 'train.csv')
sub_data_path = join(data_dir, 'test.csv')
# train.csv는 train 데이터로, test.csv는 sub 데이터로 지정한다.
data = pd.read_csv(train_data_path)
sub = pd.read_csv(sub_data_path)

In [ ]:
# 타겟 데이터인 price 컬럼은 y 변수에 따로 저장해준 후 데이터셋에서 제거한다.
# price를 로그 변환하여 정규분포를 따르게 한다.
y = data['price']
y = np.log1p(y)
del data['price']

In [ ]:
# train 데이터와 test 데이터를 pd.concat 메서드로 병합한다.
data = pd.concat((data, sub), axis=0)
data

In [ ]:
# 각 컬럼별 결측치 개수를 확인한다. 이후 타겟인 price와 상관관계가 없어 보이는
# id 컬럼을 데이터에서 제거한다.
for i in data.columns:
    print('{} : {}'.format(i, len(data.loc[pd.isnull(data[i]), i].values)))
del data['id']

In [ ]:
# date 컬럼의 각 데이터에 포함되어 있는, 쓸모없는 부분은 지워준다.
del data['date']

In [ ]:
# 각 컬럼들마다 한쪽으로 치우친 분포를 보이는 데이터가 있는지 확인하기 위해
# 서브플롯을 그려주고 seaborn의 kdeplot으로 각 컬럼들의 데이터의 그래프를 그려본다.
fig, ax = plt.subplots(9, 2, figsize=(12, 50))
count = 1
columns = data.columns
for row in range(9):
    for col in range(2):
        sns.kdeplot(data=data[columns[count]], ax=ax[row][col])
        ax[row][col].set_title(columns[count], fontsize=15)
        count += 1
        if count == 19 :
            break

In [ ]:
# 치우친 분포를 보이는 컬럼들을 skew_columns에 모아 리스트를 만든다.
skew_columns = ['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_lot15', 'sqft_living15']

# 해당 컬럼들을 하나씩 뽑아 로그 변환을 해주어 데이터 분포가 정규 분포에 가깝게 만든다.
for i in skew_columns:
    data[i] = np.log1p(data[i].values)

In [ ]:
# 학습 데이터와 테스트 데이터로 분류하기 위해 각각 x와 sub 변수를 만들어 데이터를 나눈다.
x = data.iloc[:15035, :]
sub = data.iloc[15035:, :]

In [ ]:
# 가격 예측을 위해 훈련할 모델들을 정하고 random_state를 지정해준다.
gboost = GradientBoostingRegressor(random_state=2023)
lightgbm = lgb.LGBMRegressor(random_state=2023)
rdforest = RandomForestRegressor(random_state=2023)

# 각 모델별로 모델과 모델명을 각각 키-값으로 갖는 딕셔너리를 생성하고, 리스트로 감싸준다.
models = [gboost, lightgbm, rdforest]

In [ ]:
# 모델을 학습시키고 예측값을 csv 파일로 저장하는 함수를 만든다.
def save_submission(model, x, y, sub):
    model.fit(x, y)
    prediction = model.predict(sub)
    prediction = np.expm1(prediction)
    submission_path = 'C:/Users/ZAKAR/Documents/GitHub/AIFFEL/Exploration/Quest12_EX02/submission.csv'
    submission = pd.read_csv(submission_path)
    submission['price'] = prediction
    submission_csv_path = f'{data_dir}/submission_{model}.csv'
    submission.to_csv(submission_csv_path, index=False)
    print(f'{submission_csv_path} saved!')

In [ ]:
# 각 모델별로 학습 및 예측 후 결과를 csv 파일로 저장
for model in models:
    save_submission(model, x, y, sub)